# <center><font color='red'> TME 4 - SVM </font></center>

In [45]:
import numpy as np
#from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import collections
from sklearn.svm import *
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pandas as pd

In [3]:
def plot_data(data,labels=None):
    """
    Affiche des donnees 2D
    :param data: matrice des donnees 2d
    :param labels: vecteur des labels (discrets)
    :return:
    """
    cols,marks = ["red", "green", "blue", "orange", "black", "cyan"],[".","+","*","o","x","^"]
    if labels is None:
        plt.scatter(data[:,0],data[:,1],marker="x")
        return
    for i,l in enumerate(sorted(list(set(labels.flatten())))):
        plt.scatter(data[labels==l,0],data[labels==l,1],c=cols[i],marker=marks[i])

def plot_frontiere(data,f,step=20):
    """ Trace un graphe de la frontiere de decision de f
    :param data: donnees
    :param f: fonction de decision
    :param step: pas de la grille
    :return:
    """
    grid,x,y=make_grid(data=data,step=step)
    plt.contourf(x,y,f(grid).reshape(x.shape),colors=('gray','blue'),levels=[-1,0,1])
    
def plot_frontiere_proba(data, f, step=20):
    grid,x,y=make_grid(data=data,step=step)
    plt.contourf(x,y,f(grid).reshape(x.shape),255)

def make_grid(data=None,xmin=-5,xmax=5,ymin=-5,ymax=5,step=20):
    """ Cree une grille sous forme de matrice 2d de la liste des points
    :param data: pour calcluler les bornes du graphe
    :param xmin: si pas data, alors bornes du graphe
    :param xmax:
    :param ymin:
    :param ymax:
    :param step: pas de la grille
    :return: une matrice 2d contenant les points de la grille
    """
    if data is not None:
        xmax, xmin, ymax, ymin = np.max(data[:,0]),  np.min(data[:,0]), np.max(data[:,1]), np.min(data[:,1])
    x, y =np.meshgrid(np.arange(xmin,xmax,(xmax-xmin)*1./step), np.arange(ymin,ymax,(ymax-ymin)*1./step))
    grid=np.c_[x.ravel(),y.ravel()]
    return grid, x, y

def gen_arti(centerx=1,centery=1,sigma=0.1,nbex=1000,data_type=0,epsilon=0.02):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param epsilon: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
        #melange de 2 gaussiennes
        xpos=np.random.multivariate_normal([centerx,centerx],np.diag([sigma,sigma]),nbex//2)
        xneg=np.random.multivariate_normal([-centerx,-centerx],np.diag([sigma,sigma]),nbex//2)
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centerx],np.diag([sigma,sigma]),nbex//4),np.random.multivariate_normal([-centerx,-centerx],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centerx],np.diag([sigma,sigma]),nbex//4),np.random.multivariate_normal([centerx,-centerx],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,epsilon,nbex)
    data[:,1]+=np.random.normal(0,epsilon,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

## <center><code style="background:yellow; color:black"> SVM et Grid Search </code></center>

Le module **sklearn.svm** propose une implémentation des SVMs. Sur les jeux de données des précédents TMEs (2d artificiels et reconnaissance de chiffres), explorer plusieurs noyaux (linéaire, gaussien, polynomial) et plusieurs paramètrages des noyaux. Vous étudierez en particulier les frontières de décisions et les vecteurs supports - les points dont les coefficients sont non nuls. Comment évolue le nombre de ces derniers en fonction du noyau et de son paramètrage ? Est-ce normal ? Que retrouvez vous dans le cas linéaire ? Afin de trouver les meilleurs paramètres, on opère par validation croisée sur une grille des paramètres (grid search). Pour les différents noyaux et différents nombre d’exemples d’apprentissage, opérer un grid search afin de trouver les paramètres optimaux. Tracer les courbes d’erreurs en apprentissage et en test. Les résultats sont-ils cohérents ?

In [34]:
trainx,trainy = gen_arti(nbex=1000,data_type=0,epsilon=1)
testx,testy = gen_arti(nbex=1000,data_type=0,epsilon=1)

In [50]:
parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
clf = GridSearchCV(SVC(), parameters)
clf.fit(trainx, trainy)
clf.predict(testx)
#pred = clf.predict(testx)
print(pd.DataFrame( ( clf.cv_results_) ) )
# plot_frontiere(trainx,clf.predict,200)
# plot_data(trainx,trainy)
# print("Score : ", np.where(pred == testy, 1, 0).mean())

C:\Users\wuwen\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.010272      0.007266         0.000000        0.000000       1   
1        0.015634      0.000018         0.000000        0.000000       1   
2        0.000000      0.000000         0.005203        0.007358      10   
3        0.010414      0.007364         0.000000        0.000000      10   
4        0.005208      0.007365         0.000000        0.000000     100   
5        0.000000      0.000000         0.005207        0.007363     100   
6        0.005207      0.007364         0.000000        0.000000    1000   
7        0.000000      0.000000         0.000000        0.000000    1000   
8        0.005198      0.007352         0.000000        0.000000       1   
9        0.010415      0.007364         0.000000        0.000000      10   
10       0.026044      0.007369         0.000000        0.000000     100   
11       0.208289      0.041000         0.000000        0.000000    1000   

   param_ga